# Validation
Here we will validate models against the GridClim product.

There are four scores we can calculate to verify each model 

Lets start with the **average monthly anomaly.**
This works with a mean value for the whole timeperiod in each dataset, hence we need to create these.
It is essentially collapsing the time coordinate and taking a mean.

In [ ]:
import attribution.verification
import dask
from dask.distributed import Client
import iris
import iris.plot as iplt
from importlib import reload
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
client = Client(n_workers=4)
client.amm.start()

In [ ]:
# Where do we store the data?
base_path = "/nobackup/rossby26/users/sm_erhol/extremeEventAttribution/"
# What is the file called?
cordex_name = "prAdjust_Gavle_CORDEX-ENS_rcp85_day_19710101-20181230.nc"
# Load the file
cordex_cube = iris.load_cube(os.path.join(base_path, cordex_name))

# GridClim
# What is the file called?
gc_name = "prAdjust_Gavle_SMHIGridClim_day_19710101-20181230.nc"
# Load the file
gc_cube = iris.load_cube(os.path.join(base_path, gc_name))

## Annual anomaly

In [ ]:
average_anomaly = attribution.verification.average_anomaly(cordex_cube, gc_cube)

In [ ]:
# Print the average anomaly as a percentage.
average_anomaly.data * 100

## Seasonal anomaly

In [ ]:
average_monthly_anomaly = attribution.verification.average_monthly_anomaly(
    cordex_cube, gc_cube
)

In [ ]:
# The average seasonal anomaly as a percentage.
average_monthly_anomaly.data * 100

## Seasonal correlation index?

There are essentially two indices we can calculate here, depending on hard we want to test the model.
If we want to test the models ability to simulate the average year we don't need to set the arg `climatological` since this is `True` by default.
This will correlate the average annual cycle for the model(s) and observations in each grid point.
Doing this checks how well the model captures the average annual cycle for each grid point.

By setting `climatological=False` every annual cycle is instead correlated i.e. monthly values.
This seems to be a tougher metric, and is more computationally demanding.

In [ ]:
reload(attribution.verification)

In [ ]:
seasonality_index = attribution.verification.seasonality_index(
    cordex_cube, gc_cube, kge=False
)
seasonality_index_kge = attribution.verification.seasonality_index(
    cordex_cube, gc_cube, kge=True
)

In [ ]:
seasonality_index.data

In [ ]:
seasonality_index_kge.data

## Spatial correlation

In [ ]:
pattern_index = attribution.verification.pattern_index(cordex_cube, gc_cube)

In [ ]:
pattern_index.data

## Homogenise the performance
Mapping the index to scores between 0 and 10.

In [ ]:
# Scores for index 1 and 2
indx_1_scores = attribution.verification.get_scores(
    average_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)
indx_2_scores = attribution.verification.get_scores(
    average_monthly_anomaly.data * 100,
    bins=np.arange(5, 51, 5),
    score_bins=np.arange(10, -1, -1),
)

In [ ]:
# Scores for index 3 and 4
indx_3_scores = attribution.verification.get_scores(
    seasonality_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)
indx_4_scores = attribution.verification.get_scores(
    pattern_index.data,
    bins=np.arange(0.92, 0.19, -0.08),
    score_bins=np.arange(10, -1, -1),
)

In [ ]:
# Stack the scores
data = np.stack(
    [
        indx_1_scores,
        indx_2_scores,
        indx_3_scores,
        indx_4_scores,
    ],
    axis=1,
)

In [ ]:
# fig, ax = plt.subplots(figsize=(5, 20))
# im = ax.imshow(data, aspect=0.7, vmin=0, vmax=10)
# plt.colorbar(im)

In [ ]:
# Create a dataframe of the scores.
scores_df = pd.DataFrame(data=data, columns=["Idx_1", "Idx_2", "Idx_3", "Idx_4"])
# Add the ensemble id
scores_df["ensemble_id"] = cordex_cube.coord("ensemble_id").points
scores_df = scores_df[["ensemble_id", "Idx_1", "Idx_2", "Idx_3", "Idx_4"]]

In [ ]:
# Add a sum of the scores.
scores_df["Sum_scores"] = scores_df.sum(axis=1, numeric_only=True)

In [ ]:
scores_df

With these scores we can then drop models out of the rest of the study.
In this particular case, the lowest total score is 29, which is still acceptable.
Hence, we don't drop any models based on the performance.

In [ ]:
# Save the scores to a csv.
scores_df.to_csv("./scores.csv")